In [ ]:
- reddit, get liked posts
- twitter, get liked tweets
- linkedin, get liked posts


- consolidate into single format

In [ ]:
import os

from ingest.twitter import twitter_client

page_size = 300

tweets_page = twitter_client.get_favorites(
    user_id=os.environ["TWITTER_USERNAME"], tweet_mode="extended", count=page_size
)

len(tweets_page)


In [ ]:
body.properties.text.files should be defined, instead was `undefined`.
body.properties.text.status should be defined, instead was `undefined`.
body.properties.text.id should be defined, instead was `undefined`.
body.properties.text.name should be defined, instead was `undefined`.
body.properties.text.start should be defined, instead was `undefined`.

In [ ]:
{
    "Grocery item": {
        "type": "title",
        "title": [{"type": "text", "text": {"content": "Tomatoes"}}],
    },
    "Price": {"type": "number", "number": 1.49},
    "Last ordered": {"type": "date", "date": {"start": "2021-05-11"}},
}

# import os
# import requests

# headers = {
#     "Authorization": "Bearer " + os.environ['NOTION_API_KEY'],
#     "Content-Type": "application/json",
#     "Notion-Version": "2022-06-28",
# }

# def create_page(data: dict):
#     create_url = "https://api.notion.com/v1/pages"

#     payload = {"parent": {"database_id": os.environ['NOTION_DB_ID']}, "properties": data}

#     return requests.post(create_url, headers=headers, json=payload)

# new_page = format_notion_record(all_records[1])
# res = create_page(new_page)


In [ ]:
all_records


In [ ]:
from uuid import uuid4

uuid4().hex


In [ ]:
from ingest.notion import filter_page_by_property

filter_page_by_property("Name", "test")
